In [1]:
model_path = "hf-internal-testing/Mixtral-tiny"
token_path = "mistralai/Mixtral-8x7B-v0.1"

import numpy as np
from transformers import AutoTokenizer, MixtralModel, MixtralConfig, FlaxMixtralModel
config = MixtralConfig.from_pretrained(model_path)
config.output_router_logits = True
config.output_attentions = True
config.output_hidden_states = True


/home/adrien/Documents/flax/FlaxMixtral/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = MixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

x = model(**inputs)

MixtralModel is using MixtralSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [9]:
x.hidden_states

(tensor([[[-0.4067,  1.4824,  2.1855,  ...,  2.1699, -2.6309,  1.4844],
          [-0.5459, -0.8843, -0.2935,  ...,  0.3994,  0.3154,  0.1948],
          [-1.3867,  0.1390, -0.4028,  ..., -0.7080, -0.8794, -0.7305],
          ...,
          [ 0.5400, -1.0771,  0.5771,  ..., -0.8071, -1.1641,  0.4294],
          [-0.0669, -0.6182,  1.4971,  ..., -1.1377,  0.9434, -0.6587],
          [ 0.5278,  0.4001,  0.3354,  ...,  1.1787, -0.0963, -0.7915]]],
        grad_fn=<EmbeddingBackward0>),
 tensor([[[-0.9513,  1.3742,  2.0910,  ...,  1.8151, -2.2085,  1.4657],
          [-1.0046, -1.0591, -0.1656,  ...,  0.5409,  0.4331,  0.4100],
          [-1.5400,  0.1457, -0.1894,  ..., -0.7366, -0.6917, -0.4431],
          ...,
          [ 0.4499, -1.0704,  0.5446,  ..., -0.7091, -1.0767,  0.6993],
          [-0.1403, -0.5699,  1.3213,  ..., -1.2383,  0.9743, -0.3318],
          [ 0.6120,  0.3999,  0.2614,  ...,  1.2715,  0.0878, -0.5480]]],
        grad_fn=<AddBackward0>),
 tensor([[[-1.4910,  1.2440,  

In [3]:
model = FlaxMixtralModel.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(token_path)

prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="jax")

y = model(**inputs)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


CCCCCCCCCCCCCCCC


Some weights of the model checkpoint at hf-internal-testing/Mixtral-tiny were not used when initializing FlaxMixtralModel: {('lm_head', 'kernel')}
- This IS expected if you are initializing FlaxMixtralModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxMixtralModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some of the weights of FlaxMixtralModel were initialized in float16 precision from the model checkpoint at hf-internal-testing/Mixtral-tiny:
[('embed_tokens', 'embedding'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w1', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w2', 'kernel'), ('layers', '0', 'block_sparse_moe', 'experts', '0', 'w3', 'kernel'), ('laye

CCCCCCCCCCCCCCCC


In [10]:
y.hidden_states

(Array([[[-0.40673828,  1.4824219 ,  2.1855469 , ...,  2.1699219 ,
          -2.6308594 ,  1.484375  ],
         [-0.54589844, -0.88427734, -0.29345703, ...,  0.39941406,
           0.3154297 ,  0.19482422],
         [-1.3867188 ,  0.13903809, -0.40283203, ..., -0.7080078 ,
          -0.87939453, -0.73046875],
         ...,
         [ 0.54003906, -1.0771484 ,  0.57714844, ..., -0.8071289 ,
          -1.1640625 ,  0.42944336],
         [-0.06689453, -0.61816406,  1.4970703 , ..., -1.1376953 ,
           0.9433594 , -0.6586914 ],
         [ 0.52783203,  0.40014648,  0.33544922, ...,  1.1787109 ,
          -0.09631348, -0.7915039 ]]], dtype=float32),
 Array([[[-0.9512765 ,  1.3742137 ,  2.0909903 , ...,  1.8151293 ,
          -2.2085001 ,  1.4656765 ],
         [-1.0045583 , -1.0591412 , -0.16560225, ...,  0.54093945,
           0.43313074,  0.41004956],
         [-1.5399573 ,  0.14566039, -0.18943106, ..., -0.7365563 ,
          -0.6916512 , -0.4430975 ],
         ...,
         [ 0.44989

In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.array(y.attentions[i][j][k][l]))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.array(y.attentions[0]))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.array(y.hidden_states[i][j][k]))))


In [ ]:
to_keep = []
for i in range(len(x.attentions)):
        for j in range(len(x.attentions[i])):
                for k in range(len(x.attentions[i][j])):
                        for l in range(len(x.attentions[i][j][k])):
                                to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j][k][l].detach().cpu()) - np.around(np.array(y.attentions[i][j][k][l]), 4))))

# for i in range(len(x.last_hidden_state)):
#         for j in range(len(x.last_hidden_state[i])):
#                                 to_keep.append(np.amax(np.abs(np.array(x.attentions[i][j].detach().cpu()) - np.array(y.attentions[i][j]))))

to_keep.append(np.amax(np.abs(np.array(x.attentions[0].detach().cpu()) - np.around(np.array(y.attentions[0]), 4))))


for i in range(len(x.hidden_states)):
        for j in range(len(x.hidden_states[i])):
                for k in range(len(x.hidden_states[i][j])):
                                to_keep.append(np.amax(np.abs(np.array(x.hidden_states[i][j][k].detach().cpu()) - np.around(np.array(y.hidden_states[i][j][k]), 4))))
